1. **Install YOLO Library**  
    Install the `ultralytics` library, which provides the YOLO implementation.  

In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 676.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.6/871.6 kB 6.0 MB/s eta 0:00:00


2. **Check GPU Availability**  
    Verify the availability of GPU resources using the `nvidia-smi` command.  

In [2]:
!nvidia-smi

Sun Sep 15 15:59:34 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

3. **Install and Configure Roboflow**  
    Install the `roboflow` library, authenticate using an API key, and download the dataset for PCB component detection.  

In [3]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="keLnYV1gi59z3aBkUxfz")
project = rf.workspace("classytamil").project("pcb-component-detection2")
version = project.version(1)
dataset = version.download("yolov9")
                

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 kB 1.4 MB/s eta 0:00:00
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to pcb-component-detection2-1 in yolov9:: 100%|██████████| 2352/2352 [00:00<00:00, 5972.50it/s]


4. **Import YOLO Library**  

In [4]:
from ultralytics import YOLO

5. **Disable W&B Logging**  
    Disable Weights & Biases (W&B) logging by setting the environment variable.  

In [5]:
import os

os.environ['WANDB_DISABLED'] = 'true'


6. **Check CUDA Device Count**  
    Verify the number of available CUDA devices using PyTorch.  

In [6]:
import torch
torch.cuda.device_count()

2

7. **Modify the `data.yaml` File**  
    Update the `data.yaml` file to include the correct paths for training, validation, and test images.  

In [7]:
import yaml

# Load the existing data.yaml file
yaml_path = "/kaggle/working/pcb-component-detection2-1/data.yaml"
with open(yaml_path, 'r') as file:
    data = yaml.safe_load(file)

# Update the paths (make sure these paths point to where your images are stored)
data['train'] = '/kaggle/working/pcb-component-detection2-1/train/images'
data['val'] = '/kaggle/working/pcb-component-detection2-1/valid/images'
data['test'] = '/kaggle/working/pcb-component-detection2-1/test/images'
# Save the updated data.yaml file
with open(yaml_path, 'w') as file:
    yaml.dump(data, file)

print("Updated data.yaml file with correct image paths.")

Updated data.yaml file with correct image paths.


8. **Load the YOLO Model**  
    Load the YOLO model using a pre-trained weights file (`yolov10s.pt`).  

In [8]:
model=YOLO("yolov10s.pt")

100%|██████████| 15.9M/15.9M [00:00<00:00, 144MB/s]


9. **Train the YOLO Model**  
    Train the YOLOv9 model using the custom dataset and specify the number of epochs and GPU devices.  

In [9]:
model.train(data='/kaggle/working/pcb-component-detection2-1/data.yaml', epochs=100, device='0,1')

Ultralytics YOLOv8.2.93 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
                                                      CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov10s.pt, data=/kaggle/working/pcb-component-detection2-1/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0,1, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_

100%|██████████| 755k/755k [00:00<00:00, 14.5MB/s]
2024-09-15 15:59:58,948	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-09-15 16:00:00,031	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=7

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1     36096  ultralytics.nn.modules.block.SCDown          [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1    137728  ultralytics

wandb: Tracking run with wandb version 0.17.7
wandb: W&B syncing is set to `offline` in this directory.  
wandb: Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.


Overriding model.yaml nc=80 with nc=7
Transferred 607/619 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...


...

    100/100       2.8G      2.404     0.8682       1.98         40        640: 100%|██████████| 64/64 [00:15<00:00,  4.25it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  6.98it/s]


                   all        100        723       0.99      0.983      0.992      0.654

100 epochs completed in 0.503 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 16.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 16.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.93 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
                                                      CUDA:1 (Tesla T4, 15095MiB)
YOLOv10s summary (fused): 293 layers, 8,040,378 parameters, 0 gradients, 24.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:06<00:00,  1.07it/s]


                   all        100        723      0.965       0.98      0.987      0.658
         1002-resistor         88         88      0.954      0.966      0.983      0.575
         1201-resistor         90         90       0.96          1      0.995      0.631
          195-resistor         88         88      0.955      0.964      0.961      0.559
          221-resistor         95        184      0.993      0.995      0.995      0.594
            ams1117 IC         90         90      0.967      0.986      0.994      0.755
                esp 32         93         93      0.928      0.969      0.987      0.879
            transistor         90         90      0.999      0.978      0.994      0.614
Speed: 0.2ms preprocess, 8.2ms inference, 0.0ms loss, 3.7ms postprocess per image
Results saved to runs/detect/train


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▃███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
wandb:                  lr/pg1 ▃███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
wandb:                  lr/pg2 ▃███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
wandb:        metrics/mAP50(B) ▁▅▆▆▆▆▇▇█▇▇█▇▇▇█▇▇▇█▇▇▇▇████████████████
wandb:     metrics/mAP50-95(B) ▁▁▃▄▅▅▄▆▆▆▆▆▆▆▆▆▆▇▇▆▇▇▇▇▇▇▇▇▇▇██████▇▇██
wandb:    metrics/precision(B) ▁▆▇▆▇▇▇▆▇▇▇▇▇▇▇██▇▇▇██▇▇███▇▇██▇▇██▇███▇
wandb:       metrics/recall(B) ▁▆▅▆▆▅▆▆▇��▆▇▆▇▇▆▇▆▆▇▅▇▇▇▇▇▇██▇▇██▇███▇██
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:          train/box_loss █▅▅▅▄▄▄▄▃▃▃▃▄▃▂▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▂▂▂▁
wandb:          train/cls_loss █▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:          train/dfl_loss █▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▃▂▂▂
wandb:            val/box_loss ▇█▆▆▅▅▅▅▄▃▃▄▃▃▃▄▄▃▂▃▂▂▂▂▃▂▂▂▂▂▂▂▂▂

10. **Validate the YOLO Model**  
    Load the trained YOLO model, run validation on the test/validation dataset, print evaluation metrics (precision, recall, mAP@50, and mAP@50-95), and save the results to a file.  

In [ ]:
model = YOLO('/kaggle/working/runs/detect/train/weights/best.pt')  # Path to your best model

# Run validation on the test/validation dataset
results = model.val(data='/kaggle/working/pcb-component-detection2-2/data.yaml', split='val')

# Print the evaluation metrics
print("Metrics:")
print(f"Precision per class: {results.box.p}")
print(f"Recall per class: {results.box.r}")
print(f"mAP@50 per class: {results.box.map50}")
print(f"mAP@50-95 per class: {results.box.map}")

# Save results to a file
with open('/kaggle/working/validation_results.txt', 'w') as f:
    f.write("Precision per class: " + str(results.box.p) + '\n')
    f.write("Recall per class: " + str(results.box.r) + '\n')
    f.write("mAP@50 per class: " + str(results.box.map50) + '\n')
    f.write("mAP@50-95 per class: " + str(results.box.map) + '\n')

print("Validation results saved to validation_results.txt")